# Comprender los tipos de datos en Python

La ciencia y la computación eficaces basadas en datos requieren comprender cómo se almacenan y manipulan los datos.
Este capítulo describe y contrasta cómo se manejan las matrices de datos en el propio lenguaje Python y cómo NumPy mejora esto.
Comprender esta diferencia es fundamental para comprender gran parte del material del resto del libro.

Los usuarios de Python a menudo se sienten atraídos por su facilidad de uso, una de las cuales es la escritura dinámica.
Mientras que un lenguaje de tipo estático como C o Java requiere que cada variable se declare explícitamente, un lenguaje de tipo dinámico como Python omite esta especificación. Por ejemplo, en C podrías especificar una operación particular de la siguiente manera:

```C
/* código C */
resultado entero = 0;
para(int i=0; i<100; i++){
    resultado += i;
}
```

Mientras que en Python la operación equivalente podría escribirse de esta manera:

```pitón
# código Python
resultado = 0
para i en el rango (100):
    resultado += yo
```

Observe una diferencia principal: en C, los tipos de datos de cada variable se declaran explícitamente, mientras que en Python los tipos se infieren dinámicamente. Esto significa, por ejemplo, que podemos asignar cualquier tipo de dato a cualquier variable:

```pitón
# código Python
x = 4
x = "cuatro"
```

Aquí hemos cambiado el contenido de `x` de un número entero a una cadena. Lo mismo en C conduciría (dependiendo de la configuración del compilador) a un error de compilación u otras consecuencias no deseadas:

```C
/* código C */
entero x = 4;
x = "cuatro";  // FALLA
```

Este tipo de flexibilidad es un elemento que hace que Python y otros lenguajes tipados dinámicamente sean convenientes y fáciles de usar.
Comprender *cómo* funciona esto es una parte importante del aprendizaje para analizar datos de manera eficiente y efectiva con Python.
Pero a lo que también apunta esta flexibilidad de tipos es al hecho de que las variables de Python son más que sólo sus valores; también contienen información adicional sobre el *tipo* del valor. Exploraremos esto más en las secciones siguientes.

## Un número entero de Python es más que un simple número entero

La implementación estándar de Python está escrita en C.
Esto significa que cada objeto de Python es simplemente una estructura C hábilmente disfrazada, que contiene no sólo su valor, sino también otra información. Por ejemplo, cuando definimos un número entero en Python, como `x = 10000`, `x` no es solo un número entero "sin formato". En realidad, es un puntero a una estructura C compuesta, que contiene varios valores.
Al revisar el código fuente de Python 3.10, encontramos que la definición de tipo entero (largo) efectivamente se ve así (una vez que se expanden las macros de C):

```C
estructura _objeto largo {
    ob_refcnt largo;
    PyTypeObject *ob_type;
    tamaño_t ob_tamaño;
    largo ob_digit[1];
};
```

Un único número entero en Python 3.10 en realidad contiene cuatro partes:

- `ob_refcnt`, un recuento de referencias que ayuda a Python a manejar silenciosamente la asignación y desasignación de memoria
- `ob_type`, que codifica el tipo de variable
- `ob_size`, que especifica el tamaño de los siguientes miembros de datos
- `ob_digit`, que contiene el valor entero real que esperamos que represente la variable Python

Esto significa que existe cierta sobrecarga al almacenar un número entero en Python en comparación con un lenguaje compilado como C, como se ilustra en la siguiente figura:

![Diseño de memoria entera](images/cint_vs_pyint.png)

Aquí, `PyObject_HEAD` es la parte de la estructura que contiene el recuento de referencias, el código de tipo y otras partes mencionadas anteriormente.

Observe la diferencia aquí: un entero C es esencialmente una etiqueta para una posición en la memoria cuyos bytes codifican un valor entero.
Un entero de Python es un puntero a una posición en la memoria que contiene toda la información del objeto de Python, incluidos los bytes que contienen el valor entero.
Esta información adicional en la estructura entera de Python es lo que permite codificar Python de forma tan libre y dinámica.
Sin embargo, toda esta información adicional en los tipos de Python tiene un costo, que se vuelve especialmente evidente en estructuras que combinan muchos de estos objetos.

## Una lista de Python es más que una simple lista

Consideremos ahora qué sucede cuando usamos una estructura de datos de Python que contiene muchos objetos de Python.
El contenedor multielemento mutable estándar en Python es la lista.
Podemos crear una lista de números enteros de la siguiente manera:

In [1]:
L = list(range(10))
L

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [2]:
type(L[0])

int

O, de manera similar, una lista de cadenas:

In [3]:
L2 = [str(c) for c in L]
L2

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [4]:
type(L2[0])

str

Gracias a la tipificación dinámica de Python, podemos incluso crear listas heterogéneas:

In [5]:
L3 = [True, "2", 3.0, 4]
[type(item) for item in L3]

[bool, str, float, int]

Pero esta flexibilidad tiene un costo: para permitir estos tipos flexibles, cada elemento de la lista debe contener su propio tipo, recuento de referencias y otra información. Es decir, cada elemento es un objeto Python completo.
En el caso especial de que todas las variables sean del mismo tipo, gran parte de esta información es redundante, por lo que puede ser mucho más eficiente almacenar los datos en una matriz de tipo fijo.
La diferencia entre una lista de tipo dinámico y una matriz de tipo fijo (estilo NumPy) se ilustra en la siguiente figura:

![Diseño de memoria de matriz](images/array_vs_list.png)

En el nivel de implementación, la matriz contiene esencialmente un único puntero a un bloque de datos contiguo.
La lista de Python, por otro lado, contiene un puntero a un bloque de punteros, cada uno de los cuales a su vez apunta a un objeto Python completo como el entero de Python que vimos anteriormente.
Nuevamente, la ventaja de la lista es la flexibilidad: debido a que cada elemento de la lista es una estructura completa que contiene tanto datos como información de tipo, la lista se puede completar con datos de cualquier tipo deseado.
Las matrices de tipo fijo estilo NumPy carecen de esta flexibilidad, pero son mucho más eficientes para almacenar y manipular datos.

## Matrices de tipo fijo en Python

Python ofrece varias opciones diferentes para almacenar datos en buffers de datos eficientes y de tipo fijo.
El módulo incorporado `array` (disponible desde Python 3.3) se puede utilizar para crear matrices densas de un tipo uniforme:

In [6]:
import array
L = list(range(10))
A = array.array('i', L)
A

array('i', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Aquí, `'i'` es un código de tipo que indica que el contenido es un número entero.

Sin embargo, mucho más útil es el objeto `ndarray` del paquete NumPy.
Mientras que el objeto `array` de Python proporciona un almacenamiento eficiente de datos basados ​​en matrices, NumPy agrega a estas *operaciones* eficientes sobre esos datos.
Exploraremos estas operaciones en capítulos posteriores; A continuación, te mostraré algunas formas diferentes de crear una matriz NumPy.

## Creando matrices a partir de listas de Python

Comenzaremos con la importación estándar de NumPy, bajo el alias `np`:

In [ ]:
import numpy as np

Ahora podemos usar `np.array` para crear matrices a partir de listas de Python:

In [8]:
# Integer array
np.array([1, 4, 2, 5, 3])

array([1, 4, 2, 5, 3])

Recuerde que, a diferencia de las listas de Python, las matrices NumPy solo pueden contener datos del mismo tipo.
Si los tipos no coinciden, NumPy los actualizará de acuerdo con sus reglas de promoción de tipos; aquí, los números enteros se convierten a punto flotante:

In [9]:
np.array([3.14, 4, 2, 3])

array([3.14, 4.  , 2.  , 3.  ])

Si queremos establecer explícitamente el tipo de datos de la matriz resultante, podemos usar la palabra clave `dtype`:

In [10]:
np.array([1, 2, 3, 4], dtype=np.float32)

array([1., 2., 3., 4.], dtype=float32)

Finalmente, a diferencia de las listas de Python, que siempre son secuencias unidimensionales, las matrices NumPy pueden ser multidimensionales. Aquí hay una forma de inicializar una matriz multidimensional usando una lista de listas:

In [11]:
# Nested lists result in multidimensional arrays
np.array([range(i, i + 3) for i in [2, 4, 6]])

array([[2, 3, 4],
       [4, 5, 6],
       [6, 7, 8]])

Las listas internas se tratan como filas de la matriz bidimensional resultante.

## Creando matrices desde cero

Especialmente para matrices más grandes, es más eficiente crear matrices desde cero utilizando rutinas integradas en NumPy.
A continuación se muestran varios ejemplos:

In [12]:
# Create a length-10 integer array filled with 0s
np.zeros(10, dtype=int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [13]:
# Create a 3x5 floating-point array filled with 1s
np.ones((3, 5), dtype=float)

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [14]:
# Create a 3x5 array filled with 3.14
np.full((3, 5), 3.14)

array([[3.14, 3.14, 3.14, 3.14, 3.14],
       [3.14, 3.14, 3.14, 3.14, 3.14],
       [3.14, 3.14, 3.14, 3.14, 3.14]])

In [15]:
# Create an array filled with a linear sequence
# starting at 0, ending at 20, stepping by 2
# (this is similar to the built-in range function)
np.arange(0, 20, 2)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [16]:
# Create an array of five values evenly spaced between 0 and 1
np.linspace(0, 1, 5)

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [17]:
# Create a 3x3 array of uniformly distributed
# pseudorandom values between 0 and 1
np.random.random((3, 3))

array([[0.09610171, 0.88193001, 0.70548015],
       [0.35885395, 0.91670468, 0.8721031 ],
       [0.73237865, 0.09708562, 0.52506779]])

In [18]:
# Create a 3x3 array of normally distributed pseudorandom
# values with mean 0 and standard deviation 1
np.random.normal(0, 1, (3, 3))

array([[-0.46652655, -0.59158776, -1.05392451],
       [-1.72634268,  0.03194069, -0.51048869],
       [ 1.41240208,  1.77734462, -0.43820037]])

In [19]:
# Create a 3x3 array of pseudorandom integers in the interval [0, 10)
np.random.randint(0, 10, (3, 3))

array([[4, 3, 8],
       [6, 5, 0],
       [1, 1, 4]])

In [20]:
# Create a 3x3 identity matrix
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [21]:
# Create an uninitialized array of three integers; the values will be
# whatever happens to already exist at that memory location
np.empty(3)

array([1., 1., 1.])

## Tipos de datos estándar NumPy

Las matrices NumPy contienen valores de un solo tipo, por lo que es importante tener un conocimiento detallado de esos tipos y sus limitaciones.
Debido a que NumPy está construido en C, los tipos resultarán familiares para los usuarios de C, Fortran y otros lenguajes relacionados.

Los tipos de datos NumPy estándar se enumeran en la siguiente tabla.
Tenga en cuenta que al construir una matriz, se pueden especificar usando una cadena:

```pitón
np.zeros(10, dtype='int16')
```

O usando el objeto NumPy asociado:

```pitón
np.zeros(10, dtipo=np.int16)
```

| Tipo de datos | Descripción |
|-------------|-------------|
| `bool_` | Booleano (Verdadero o Falso) almacenado como un byte |
| `int_` | Tipo entero predeterminado (igual que C `long`; normalmente `int64` o `int32`)| 
| `intc` | Idéntico a C `int` (normalmente `int32` o `int64`)| 
| `intp` | Entero utilizado para la indexación (igual que C `ssize_t`; normalmente `int32` o `int64`)| 
| `int8` | Byte (–128 ​​a 127)| 
| `int16` | Entero (–32768 a 32767)|
| `int32` | Entero (–2147483648 a 2147483647)|
| `int64` | Entero (–9223372036854775808 a 9223372036854775807)| 
| `uint8` | Entero sin signo (0 a 255)| 
| `uint16` | Entero sin signo (0 a 65535)| 
| `uint32` | Entero sin signo (0 a 4294967295)| 
| `uint64` | Entero sin signo (0 a 18446744073709551615)| 
| `flotar_` | Taquigrafía de `float64`| 
| `flotador16` | Flotador de media precisión: bit de signo, exponente de 5 bits, mantisa de 10 bits| 
| `flotante32` | Flotador de precisión simple: bit de signo, exponente de 8 bits, mantisa de 23 bits| 
| `flotante64` | Flotador de doble precisión: bit de signo, exponente de 11 bits, mantisa de 52 bits| 
| `complejo_` | Taquigrafía de `complejo128`| 
| `complejo64` | Número complejo, representado por dos flotantes de 32 bits| 
| `complejo128`| Número complejo, representado por dos flotantes de 64 bits|

Es posible una especificación de tipo más avanzada, como especificar números big- o little-endian; Para obtener más información, consulte la [documentación de NumPy] (http://numpy.org/).
NumPy también admite tipos de datos compuestos, que se tratarán en [Datos estructurados: matrices estructuradas de NumPy] (02.09-Datos-estructurados-NumPy.ipynb).